# The Price is Right

## Week 8 Order of Play

Day 1: Modal.com and SpecialistAgent  
Day 2: RAG, FrontierAgent, Ensemble Agent  
Day 3: ScannerAgent, MessengerAgent  
Day 4: AutonomousPlannerAgent and DealAgentFramework  
Day 5: The Price Is Right Finale


Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection
import logging
import requests
load_dotenv(override=True)
openai = OpenAI()
MODEL = 'gpt-5-mini'

In [2]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 3/3 [00:38<00:00, 12.72s/it]


In [3]:
len(deals)

30

In [4]:
deals[10].describe()

"Title: Koah 64GB SDHC Class-10 Memory Card for $4 + free shipping\nDetails: That's a $16 low and a very good price for a 64GB card in general. Buy Now at eBay\nFeatures: \nURL: https://www.dealnews.com/products/Koah/Koah-64-GB-SDHC-Class-10-Memory-Card/497468.html?iref=rss-c39"

### We are going to ask GPT-5-mini to summarize deals and identify their price

In [5]:
SYSTEM_PROMPT = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
"""

USER_PROMPT_PREFIX = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a short paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""

USER_PROMPT_SUFFIX = "\n\nInclude exactly 5 deals, no more."

In [6]:
# this makes a suitable user prompt given scraped deals

def make_user_prompt(scraped):
    user_prompt = USER_PROMPT_PREFIX
    user_prompt += '\n\n'.join([scrape.describe() for scrape in scraped])
    user_prompt += USER_PROMPT_SUFFIX
    return user_prompt

In [7]:
# Let's create a user prompt for the deals we just scraped, and look at how it begins

user_prompt = make_user_prompt(deals)
print(user_prompt[:2000])
messages = [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": user_prompt}]

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a short paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Big Game Super Specials at B&H Photo Video: Up to $3,000 off + free shipping
Details: Shop new TVs, projectors, speakers, streaming devices, and more in preparation for the big game. You can save up to $2,200 on Samsung TVs, up to $1,600 off LG TVs, up to $400 off JBL speakers, and up to a whopping $3,000 off home theater projectors. All orders ship for free, too. Shop Now at B

In [8]:
response = openai.chat.completions.parse(model=MODEL, messages=messages, response_format=DealSelection, reasoning_effort="minimal")
results = response.choices[0].message.parsed
results

DealSelection(deals=[Deal(product_description='Samsung 43" The Frame LS03F is a 4K QLED smart TV designed to double as wall art when not in use. It uses QLED color technology for vivid picture quality and supports standard smart TV features and apps. The certified refurbished unit includes a 2-year Allstate warranty, making it a good value for buyers who want premium design and a long warranty at a reduced price.', price=570.0, url='https://www.dealnews.com/products/Samsung/Samsung-43-The-Frame-LS03-F-4-K-QLED-Smart-TV/497459.html?iref=rss-c142'), Deal(product_description='Apple Mac mini (2024) with the M4 chip is a compact desktop offering Apple’s latest silicon performance paired with 16GB of RAM and a 256GB SSD. It runs macOS and provides strong multicore CPU performance, efficient power use, and support for multiple displays and modern connectivity. This open-box unit gives near-new performance in a small-form workstation suitable for creative work, development, or general desktop 

In [9]:
for deal in results.deals:
    print(deal.product_description)
    print(deal.price)
    print(deal.url)
    print()


Samsung 43" The Frame LS03F is a 4K QLED smart TV designed to double as wall art when not in use. It uses QLED color technology for vivid picture quality and supports standard smart TV features and apps. The certified refurbished unit includes a 2-year Allstate warranty, making it a good value for buyers who want premium design and a long warranty at a reduced price.
570.0
https://www.dealnews.com/products/Samsung/Samsung-43-The-Frame-LS03-F-4-K-QLED-Smart-TV/497459.html?iref=rss-c142

Apple Mac mini (2024) with the M4 chip is a compact desktop offering Apple’s latest silicon performance paired with 16GB of RAM and a 256GB SSD. It runs macOS and provides strong multicore CPU performance, efficient power use, and support for multiple displays and modern connectivity. This open-box unit gives near-new performance in a small-form workstation suitable for creative work, development, or general desktop use.
487.0
https://www.dealnews.com/products/Apple/Apple-Mac-mini-M4-Desktop-2024-w-256-G

In [10]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [11]:
from agents.scanner_agent import ScannerAgent

In [12]:
agent = ScannerAgent()
result = agent.scan()

INFO:root:[Scanner Agent] Scanner Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is ready
INFO:root:[Scanner Agent] Scanner Agent is about to fetch deals from RSS feed
INFO:root:[Scanner Agent] Scanner Agent received 30 deals not already scraped
INFO:root:[Scanner Agent] Scanner Agent is calling OpenAI using Structured Outputs
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Scanner Agent] Scanner Agent received 5 selected deals with price>0 from OpenAI


In [13]:
result

DealSelection(deals=[Deal(product_description='JBL Bar 750W 5.1‑channel Dolby Atmos soundbar system with a wireless subwoofer. This open‑box unit delivers multi‑channel immersive audio and deep bass suitable for home theater setups, supporting Dolby Atmos height effects and a dedicated wireless sub for low‑end impact. The package is ideal for adding cinematic sound to a TV or media room without separate AV receiver complexity.', price=367.0, url='https://www.dealnews.com/products/JBL/JBL-Bar-500-MK2-750-W-5-1-Ch-Dolby-Atmos-Soundbar-w-Wireless-Subwoofer/497460.html?iref=rss-c142'), Deal(product_description='Samsung 43" The Frame LS03F 4K QLED Smart TV (certified refurbished) that combines a 4K QLED panel with an art‑display mode to show artwork or photos when not in use. It offers smart TV features, QLED color performance, and a slim frame designed to look like wall art, making it suitable for living rooms or bedrooms where aesthetics matter as well as picture quality.', price=570.0, u

### Introducing Pushover

Pushover is a nifty tool for sending Push Notifications to your phone.

It's super easy to set up and install!

Simply visit https://pushover.net/ and click 'Login or Signup' on the top right to sign up for a free account, and create your API keys.

Once you've signed up, on the home screen, click "Create an Application/API Token", and give it any name (like AIEngineer) and click Create Application.

Then add 2 lines to your `.env` file:

PUSHOVER_USER=_put the key that's on the top right of your Pushover home screen and probably starts with a u_  
PUSHOVER_TOKEN=_put the key when you click into your new application called Agents (or whatever) and probably starts with an a_

Remember to save your `.env` file, and run `load_dotenv(override=True)` after saving, to set your environment variables.

Finally, click "Add Phone, Tablet or Desktop" to install on your phone.

In [25]:
load_dotenv(override=True)

True

In [26]:
pushover_user = os.getenv('PUSHOVER_USER')
pushover_token = os.getenv('PUSHOVER_TOKEN')
pushover_url = "https://api.pushover.net/1/messages.json"

In [27]:
if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

Pushover user found and starts with u
Pushover token found and starts with a


In [28]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [29]:
push("MASSIVE DEAL!!")

Push: MASSIVE DEAL!!


In [30]:
from agents.messaging_agent import MessagingAgent

agent = MessagingAgent()
agent.push("SUCH A MASSIVE DEAL!!")

INFO:root:[Messaging Agent] Messaging Agent is initializing
INFO:root:[Messaging Agent] Messaging Agent has initialized Pushover and GPT5.1
INFO:root:[Messaging Agent] Messaging Agent is sending a push notification


In [31]:
agent.notify("A special deal on Sumsung 60 inch LED TV going at a great bargain", 300, 1000, "www.samsung.com")

INFO:root:[Messaging Agent] Messaging Agent is using GPT5.1 to craft the message
09:21:56 - LiteLLM:INFO: utils.py:3421 - 
LiteLLM completion() model= gpt-5.1; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-5.1; provider = openai
09:21:58 - LiteLLM:INFO: utils.py:1302 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:root:[Messaging Agent] Messaging Agent is sending a push notification
INFO:root:[Messaging Agent] Messaging Agent has completed
